# 构建回归模型：线性回归和多项式回归模型


## 南瓜定价的线性回归和多项式回归 - 第三课
<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>信息图作者：Dasani Madipalli</figcaption>


<!--![信息图作者：Dasani Madipalli](../../../../../../translated_images/linear-polynomial.5523c7cb6576ccab0fecbd0e3505986eb2d191d9378e785f82befcf3a578a6e7.zh.png){width="800"}-->

#### 介绍

到目前为止，你已经通过南瓜定价数据集的样本数据了解了什么是回归，并将在整个课程中使用该数据集。你还使用了 `ggplot2` 进行了可视化。💪

现在你已经准备好深入学习机器学习中的回归。在本课中，你将进一步了解两种回归类型：*基本线性回归* 和 *多项式回归*，以及这些技术背后的一些数学原理。

> 在整个课程中，我们假设学生的数学知识较少，并努力使内容对来自其他领域的学生更易理解，因此请注意笔记、🧮 数学提示、图表以及其他学习工具，这些都将帮助你更好地理解。

#### 准备工作

提醒一下，你正在加载这些数据以便对其进行分析。

-   什么时候是购买南瓜的最佳时间？

-   一箱迷你南瓜的价格大概是多少？

-   我应该选择半蒲式耳篮子还是 1 1/9 蒲式耳箱来购买？让我们继续深入挖掘这些数据。

在上一课中，你创建了一个 `tibble`（数据框的一种现代化形式），并用原始数据集的一部分填充它，同时将价格标准化为以蒲式耳为单位。然而，通过这种方式，你只能收集到大约 400 个数据点，并且仅限于秋季月份。也许通过进一步清理数据，我们可以获得更多细节？我们拭目以待... 🕵️‍♀️

完成此任务需要以下包：

-   `tidyverse`： [tidyverse](https://www.tidyverse.org/) 是一个 [R 包集合](https://www.tidyverse.org/packages)，旨在让数据科学更快、更简单、更有趣！

-   `tidymodels`： [tidymodels](https://www.tidymodels.org/) 框架是一个 [包集合](https://www.tidymodels.org/packages)，用于建模和机器学习。

-   `janitor`： [janitor 包](https://github.com/sfirke/janitor) 提供了一些简单的小工具，用于检查和清理脏数据。

-   `corrplot`： [corrplot 包](https://cran.r-project.org/web/packages/corrplot/vignettes/corrplot-intro.html) 提供了一个可视化的相关矩阵探索工具，支持自动变量重新排序，以帮助发现变量之间隐藏的模式。

你可以通过以下命令安装这些包：

`install.packages(c("tidyverse", "tidymodels", "janitor", "corrplot"))`

下面的脚本会检查你是否安装了完成本模块所需的包，并在缺少时为你安装它们。


In [ ]:
suppressWarnings(if (!require("pacman")) install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, janitor, corrplot)

我们稍后会加载这些很棒的包，并将它们在当前的 R 会话中可用。（这只是为了说明，`pacman::p_load()` 已经帮你完成了这一步）

## 1. 线性回归线

正如你在第一课中学到的，线性回归的目标是能够绘制一条*最佳拟合线*，以便：

-   **展示变量关系**。展示变量之间的关系

-   **进行预测**。准确预测新数据点在这条线上的位置

为了绘制这种类型的线，我们使用一种统计技术，称为**最小二乘回归**。`最小二乘`的意思是回归线周围的所有数据点的误差平方后相加。理想情况下，这个最终的总和应该尽可能小，因为我们希望误差数量较低，也就是`最小二乘`。因此，最佳拟合线就是使误差平方和最小的那条线——这就是*最小二乘回归*的名称由来。

我们这样做是因为我们希望拟合一条与所有数据点的累计距离最小的线。在加总之前，我们会对误差进行平方，因为我们关心的是误差的大小，而不是方向。

> **🧮 数学公式**
>
> 这条线，称为*最佳拟合线*，可以用[一个公式](https://en.wikipedia.org/wiki/Simple_linear_regression)表示：
>
>     Y = a + bX
>
> `X` 是`解释变量`或`预测变量`，`Y` 是`因变量`或`结果变量`。线的斜率是 `b`，而 `a` 是 y 截距，指的是当 `X = 0` 时 `Y` 的值。
>

> ![](../../../../../../2-Regression/3-Linear/solution/images/slope.png "slope = $y/x$")
    信息图由 Jen Looper 制作
>
> 首先，计算斜率 `b`。
>
> 换句话说，参考我们的南瓜数据的原始问题：“按月份预测每蒲式耳南瓜的价格”，`X` 表示价格，`Y` 表示销售月份。
>
> ![](../../../../../../translated_images/calculation.989aa7822020d9d0ba9fc781f1ab5192f3421be86ebb88026528aef33c37b0d8.zh.png)
    信息图由 Jen Looper 制作
> 
> 计算 Y 的值。如果你支付大约 \$4，那一定是四月！
>
> 计算这条线的数学公式必须展示线的斜率，这也取决于截距，即当 `X = 0` 时 `Y` 的位置。
>
> 你可以在 [Math is Fun](https://www.mathsisfun.com/data/least-squares-regression.html) 网站上观察这些值的计算方法。也可以访问[这个最小二乘计算器](https://www.mathsisfun.com/data/least-squares-calculator.html)，看看数值如何影响这条线。

是不是没那么可怕？🤓

#### 相关性

还有一个需要理解的术语是给定 X 和 Y 变量之间的**相关系数**。使用散点图，你可以快速可视化这个系数。数据点整齐排列成一条线的图表具有高相关性，而数据点在 X 和 Y 之间随意分布的图表则相关性较低。

一个好的线性回归模型应该是使用最小二乘回归方法和回归线时，相关系数较高（接近 1 而不是 0）的模型。


## **2. 与数据共舞：创建用于建模的数据框**

<p >
   <img src="../../images/janitor.jpg"
   width="700"/>
   <figcaption>插画作者：@allison_horst</figcaption>


<!--![插画作者：\@allison_horst](../../../../../../translated_images/janitor.e4a77dd3d3e6a32e25327090b8a9c00dc7cf459c44fa9f184c5ecb0d48ce3794.zh.jpg){width="700"}-->


加载所需的库和数据集。将数据转换为包含数据子集的数据框：

-   仅获取按蒲式耳定价的南瓜数据

-   将日期转换为月份

-   计算价格为高价和低价的平均值

-   将价格转换为反映按蒲式耳数量定价的形式

> 我们在[上一课](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/2-Data/solution/lesson_2-R.ipynb)中已经涵盖了这些步骤。


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)
library(lubridate)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n = 5)

出于纯粹冒险的精神，让我们探索 [`janitor package`](../../../../../../2-Regression/3-Linear/solution/R/github.com/sfirke/janitor)，它提供了简单的函数来检查和清理脏数据。例如，让我们看看我们数据的列名：


In [ ]:
# Return column names
pumpkins %>% 
  names()

🤔 我们可以做得更好。让我们通过使用 `janitor::clean_names` 将这些列名转换为 [snake_case](https://en.wikipedia.org/wiki/Snake_case) 约定来使它们成为 `friendR`。要了解有关此函数的更多信息：`?clean_names`


In [ ]:
# Clean names to the snake_case convention
pumpkins <- pumpkins %>% 
  clean_names(case = "snake")

# Return column names
pumpkins %>% 
  names()

非常整洁 🧹！现在，像上一节课一样，用 `dplyr` 来与数据共舞吧！💃


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(variety, city_name, package, low_price, high_price, date)



# Extract the month from the dates to a new column
pumpkins <- pumpkins %>%
  mutate(date = mdy(date),
         month = month(date)) %>% 
  select(-date)



# Create a new column for average Price
pumpkins <- pumpkins %>% 
  mutate(price = (low_price + high_price)/2)


# Retain only pumpkins with the string "bushel"
new_pumpkins <- pumpkins %>% 
  filter(str_detect(string = package, pattern = "bushel"))


# Normalize the pricing so that you show the pricing per bushel, not per 1 1/9 or 1/2 bushel
new_pumpkins <- new_pumpkins %>% 
  mutate(price = case_when(
    str_detect(package, "1 1/9") ~ price/(1.1),
    str_detect(package, "1/2") ~ price*2,
    TRUE ~ price))

# Relocate column positions
new_pumpkins <- new_pumpkins %>% 
  relocate(month, .before = variety)


# Display the first 5 rows
new_pumpkins %>% 
  slice_head(n = 5)

干得好！👌 你现在拥有一个干净整洁的数据集，可以用来构建新的回归模型！

画个散点图怎么样？


In [ ]:
# Set theme
theme_set(theme_light())

# Make a scatter plot of month and price
new_pumpkins %>% 
  ggplot(mapping = aes(x = month, y = price)) +
  geom_point(size = 1.6)


散点图提醒我们，我们只有从八月到十二月的月度数据。我们可能需要更多的数据才能以线性方式得出结论。

让我们再看看我们的建模数据：


In [ ]:
# Display first 5 rows
new_pumpkins %>% 
  slice_head(n = 5)

如果我们想根据`city`或`package`列（它们是字符类型）来预测南瓜的`price`，该怎么办？或者更简单地说，我们如何找到`package`和`price`之间的相关性（这要求两个输入都为数值类型）呢？🤷🤷

机器学习模型在处理数值特征时效果最佳，而不是文本值，因此通常需要将分类特征转换为数值表示。

这意味着我们需要找到一种方法来重新格式化我们的预测变量，使其更容易被模型有效利用，这个过程被称为`特征工程`。


## 3. 为建模预处理数据，使用 recipes 👩‍🍳👨‍🍳

将预测变量重新格式化以便模型更有效使用的活动被称为`特征工程`。

不同的模型对数据预处理有不同的要求。例如，最小二乘法需要对`分类变量`（如月份、品种和城市名称）进行`编码`。这通常涉及将包含`分类值`的列`转换`为一个或多个`数值列`，以替代原始列。

例如，假设你的数据包含以下分类特征：

|  city   |
|:-------:|
| Denver  |
| Nairobi |
|  Tokyo  |

你可以应用*序数编码*，为每个类别替换一个唯一的整数值，如下所示：

| city |
|:----:|
|  0   |
|  1   |
|  2   |

这就是我们将对数据进行的操作！

在本节中，我们将探索另一个令人惊叹的 Tidymodels 包：[recipes](https://tidymodels.github.io/recipes/) - 它专为在训练模型**之前**帮助你预处理数据而设计。本质上，recipe 是一个对象，用于定义对数据集应用哪些步骤，以使其为建模做好准备。

现在，让我们创建一个 recipe，通过为预测变量列中的所有观测值替换唯一整数，来为建模准备数据：


In [ ]:
# Specify a recipe
pumpkins_recipe <- recipe(price ~ ., data = new_pumpkins) %>% 
  step_integer(all_predictors(), zero_based = TRUE)


# Print out the recipe
pumpkins_recipe

太棒了！👏 我们刚刚创建了第一个配方，它指定了一个结果（价格）及其对应的预测变量，并且所有预测变量列都被编码为一组整数 🙌！让我们快速分解一下：

- 调用 `recipe()` 并使用公式告诉配方变量的*角色*，以 `new_pumpkins` 数据作为参考。例如，`price` 列被分配了 `outcome` 角色，而其余列被分配了 `predictor` 角色。

- `step_integer(all_predictors(), zero_based = TRUE)` 指定所有预测变量都应转换为一组整数，编号从 0 开始。

我们相信你可能会有这样的想法：“这太酷了！！但如果我需要确认这些配方确实按照我的预期在工作怎么办？🤔”

这是一个很棒的想法！你看，一旦定义了配方，你可以估算实际预处理数据所需的参数，然后提取处理后的数据。通常在使用 Tidymodels 时不需要这样做（我们稍后会看到常规方法 -> `workflows`），但当你想进行某种合理性检查以确认配方是否按预期工作时，这会非常有用。

为此，你需要两个额外的动词：`prep()` 和 `bake()`。一如既往，我们的小 R 朋友由 [`Allison Horst`](https://github.com/allisonhorst/stats-illustrations) 创作，帮助你更好地理解这一点！

<p >
   <img src="../../images/recipes.png"
   width="550"/>
   <figcaption>插图作者：@allison_horst</figcaption>


[`prep()`](https://recipes.tidymodels.org/reference/prep.html)：从训练集估算所需参数，这些参数可以稍后应用于其他数据集。例如，对于给定的预测变量列，哪个观测值会被分配为整数 0、1、2 等。

[`bake()`](https://recipes.tidymodels.org/reference/bake.html)：使用已准备好的配方并将操作应用于任何数据集。

话虽如此，让我们准备并应用配方，真正确认在底层，预测变量列会先被编码，然后再拟合模型。


In [ ]:
# Prep the recipe
pumpkins_prep <- prep(pumpkins_recipe)

# Bake the recipe to extract a preprocessed new_pumpkins data
baked_pumpkins <- bake(pumpkins_prep, new_data = NULL)

# Print out the baked data set
baked_pumpkins %>% 
  slice_head(n = 10)

哇哦！🥳 处理后的数据 `baked_pumpkins` 的所有预测变量都已编码，这确认了我们定义的预处理步骤（作为我们的配方）确实可以如预期般工作。这虽然让数据更难阅读，但对 Tidymodels 来说却更加易于理解！花点时间找出哪些观测值已被映射到对应的整数。

另外值得一提的是，`baked_pumpkins` 是一个数据框，我们可以在其上进行计算。

例如，我们可以尝试在数据中的两个点之间找到一个良好的相关性，以便可能构建一个优秀的预测模型。我们将使用函数 `cor()` 来完成此操作。输入 `?cor()` 以了解更多关于该函数的信息。


In [ ]:
# Find the correlation between the city_name and the price
cor(baked_pumpkins$city_name, baked_pumpkins$price)

# Find the correlation between the package and the price
cor(baked_pumpkins$package, baked_pumpkins$price)


事实证明，城市和价格之间的相关性较弱。然而，套餐和价格之间的相关性稍强一些。这很合理，对吧？通常来说，生产箱越大，价格越高。

既然如此，我们也可以尝试使用 `corrplot` 包来可视化所有列的相关性矩阵。


In [ ]:
# Load the corrplot package
library(corrplot)

# Obtain correlation matrix
corr_mat <- cor(baked_pumpkins %>% 
                  # Drop columns that are not really informative
                  select(-c(low_price, high_price)))

# Make a correlation plot between the variables
corrplot(corr_mat, method = "shade", shade.col = NA, tl.col = "black", tl.srt = 45, addCoef.col = "black", cl.pos = "n", order = "original")

🤩🤩 好得多。

现在可以问这个数据的一个好问题是：'`给定一个南瓜包，我可以预期它的价格是多少？`' 让我们直接开始吧！

> 注意：当你使用 **`new_data = NULL`** 对预处理过的配方 **`pumpkins_prep`** 进行 **`bake()`** 时，你会提取处理过的（即编码后的）训练数据。如果你有另一个数据集，例如测试集，并希望查看配方如何对其进行预处理，你只需使用 **`new_data = test_set`** 对 **`pumpkins_prep`** 进行 bake。

## 4. 构建线性回归模型

<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Dasani Madipalli 制作的信息图</figcaption>


<!--![Dasani Madipalli 制作的信息图](../../../../../../translated_images/linear-polynomial.5523c7cb6576ccab0fecbd0e3505986eb2d191d9378e785f82befcf3a578a6e7.zh.png){width="800"}-->


现在我们已经构建了一个配方，并确认数据将被适当预处理，接下来让我们构建一个回归模型来回答这个问题：`我可以预期某个南瓜包装的价格是多少？`

#### 使用训练集训练线性回归模型

正如你可能已经猜到的，*price* 列是 `结果` 变量，而 *package* 列是 `预测` 变量。

为此，我们首先将数据分割为训练集（占80%）和测试集（占20%），然后定义一个配方，将预测变量列编码为一组整数，接着构建一个模型规范。我们不会准备和烘焙配方，因为我们已经知道它会按预期预处理数据。


In [ ]:
set.seed(2056)
# Split the data into training and test sets
pumpkins_split <- new_pumpkins %>% 
  initial_split(prop = 0.8)


# Extract training and test data
pumpkins_train <- training(pumpkins_split)
pumpkins_test <- testing(pumpkins_split)



# Create a recipe for preprocessing the data
lm_pumpkins_recipe <- recipe(price ~ package, data = pumpkins_train) %>% 
  step_integer(all_predictors(), zero_based = TRUE)



# Create a linear model specification
lm_spec <- linear_reg() %>% 
  set_engine("lm") %>% 
  set_mode("regression")

干得好！现在我们已经有了一个配方和模型规范，我们需要找到一种方法将它们打包成一个对象，该对象将首先对数据进行预处理（幕后完成prep+bake），然后在预处理后的数据上拟合模型，同时还支持潜在的后处理活动。这是不是让你更安心了！🤩

在Tidymodels中，这个方便的对象叫做[`workflow`](https://workflows.tidymodels.org/)，它可以方便地包含你的建模组件！这在*Python*中我们称之为*管道*。

那么，让我们把所有东西打包到一个workflow中吧！📦


In [ ]:
# Hold modelling components in a workflow
lm_wf <- workflow() %>% 
  add_recipe(lm_pumpkins_recipe) %>% 
  add_model(lm_spec)

# Print out the workflow
lm_wf

顺便提一下，工作流程可以像模型一样进行适配或训练。


In [ ]:
# Train the model
lm_wf_fit <- lm_wf %>% 
  fit(data = pumpkins_train)

# Print the model coefficients learned 
lm_wf_fit

从模型输出中，我们可以看到训练过程中学习到的系数。它们表示最佳拟合线的系数，该线使实际变量与预测变量之间的总体误差最小化。

#### 使用测试集评估模型性能

是时候看看模型的表现了 📏！我们该怎么做呢？

现在我们已经训练了模型，可以使用 `parsnip::predict()` 对测试集进行预测。然后，我们可以将这些预测值与实际标签值进行比较，以评估模型的效果（好或不好）。

让我们从对测试集进行预测开始，然后将预测结果与测试集绑定在一起。


In [ ]:
# Make predictions for the test set
predictions <- lm_wf_fit %>% 
  predict(new_data = pumpkins_test)


# Bind predictions to the test set
lm_results <- pumpkins_test %>% 
  select(c(package, price)) %>% 
  bind_cols(predictions)


# Print the first ten rows of the tibble
lm_results %>% 
  slice_head(n = 10)

是的，你刚刚训练了一个模型并用它进行了预测！🔮 它表现如何呢？让我们来评估模型的性能吧！

在Tidymodels中，我们使用 `yardstick::metrics()` 来完成这一任务！对于线性回归，我们重点关注以下指标：

-   `均方根误差 (RMSE)`：即[均方误差 (MSE)](https://en.wikipedia.org/wiki/Mean_squared_error)的平方根。它提供了一个绝对指标，单位与标签一致（在这个例子中是南瓜的价格）。值越小，模型越好（简单来说，它表示预测值平均偏差的价格范围）。

-   `决定系数（通常称为R平方或R2）`：一个相对指标，值越高，模型拟合效果越好。实际上，这个指标表示模型能够解释预测值与实际标签值之间方差的程度。


In [ ]:
# Evaluate performance of linear regression
metrics(data = lm_results,
        truth = price,
        estimate = .pred)

模型性能下降了。让我们通过可视化包裹和价格的散点图来看看是否能获得更好的指示，然后使用预测结果叠加一条最佳拟合线。

这意味着我们需要准备并处理测试集，以便对包裹列进行编码，然后将其与模型生成的预测结果绑定在一起。


In [ ]:
# Encode package column
package_encode <- lm_pumpkins_recipe %>% 
  prep() %>% 
  bake(new_data = pumpkins_test) %>% 
  select(package)


# Bind encoded package column to the results
lm_results <- lm_results %>% 
  bind_cols(package_encode %>% 
              rename(package_integer = package)) %>% 
  relocate(package_integer, .after = package)


# Print new results data frame
lm_results %>% 
  slice_head(n = 5)


# Make a scatter plot
lm_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_line(aes(y = .pred), color = "orange", size = 1.2) +
  xlab("package")
  


很棒！正如你所看到的，线性回归模型并不能很好地概括包裹与其对应价格之间的关系。

🎃 恭喜你，你刚刚创建了一个可以帮助预测几种南瓜价格的模型。你的节日南瓜田会非常漂亮。但你可能可以创建一个更好的模型！

## 5. 构建一个多项式回归模型

<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>信息图由 Dasani Madipalli 制作</figcaption>


<!--![信息图由 Dasani Madipalli 制作](../../../../../../translated_images/linear-polynomial.5523c7cb6576ccab0fecbd0e3505986eb2d191d9378e785f82befcf3a578a6e7.zh.png){width="800"}-->


有时候，我们的数据可能并不存在线性关系，但我们仍然希望预测结果。这时，多项式回归可以帮助我们对更复杂的非线性关系进行预测。

以我们的南瓜数据集中的包装和价格关系为例。虽然有时变量之间存在线性关系——比如南瓜的体积越大，价格越高——但有时这些关系无法用一个平面或直线来表示。

> ✅ 这里有[更多使用多项式回归的数据示例](https://online.stat.psu.edu/stat501/lesson/9/9.8)
>
> 再次看看之前图中品种与价格的关系。这个散点图看起来是否一定应该用一条直线来分析？可能并不是。在这种情况下，你可以尝试使用多项式回归。
>
> ✅ 多项式是可能包含一个或多个变量和系数的数学表达式

#### 使用训练集训练一个多项式回归模型

多项式回归会创建一条*曲线*，以更好地拟合非线性数据。

让我们看看多项式模型是否能在预测中表现得更好。我们将遵循与之前类似的步骤：

-   创建一个配方，指定对数据进行建模前需要执行的预处理步骤，例如：对预测变量进行编码并计算次数为 *n* 的多项式

-   构建一个模型规范

-   将配方和模型规范打包到一个工作流中

-   通过拟合工作流来创建模型

-   评估模型在测试数据上的表现

让我们开始吧！


In [ ]:
# Specify a recipe
poly_pumpkins_recipe <-
  recipe(price ~ package, data = pumpkins_train) %>%
  step_integer(all_predictors(), zero_based = TRUE) %>% 
  step_poly(all_predictors(), degree = 4)


# Create a model specification
poly_spec <- linear_reg() %>% 
  set_engine("lm") %>% 
  set_mode("regression")


# Bundle recipe and model spec into a workflow
poly_wf <- workflow() %>% 
  add_recipe(poly_pumpkins_recipe) %>% 
  add_model(poly_spec)


# Create a model
poly_wf_fit <- poly_wf %>% 
  fit(data = pumpkins_train)


# Print learned model coefficients
poly_wf_fit

  

#### 评估模型性能

👏👏你已经构建了一个多项式模型，现在让我们在测试集上进行预测吧！


In [ ]:
# Make price predictions on test data
poly_results <- poly_wf_fit %>% predict(new_data = pumpkins_test) %>% 
  bind_cols(pumpkins_test %>% select(c(package, price))) %>% 
  relocate(.pred, .after = last_col())


# Print the results
poly_results %>% 
  slice_head(n = 10)

Woo-hoo，让我们使用 `yardstick::metrics()` 来评估模型在 test_set 上的表现。


In [ ]:
metrics(data = poly_results, truth = price, estimate = .pred)

🤩🤩 表现更出色。

`rmse` 从大约 7 降至大约 3，这表明实际价格与预测价格之间的误差减少了。你可以*粗略地*理解为平均而言，错误预测的误差大约为 \$3。`rsq` 从大约 0.4 增加到 0.8。

所有这些指标都表明多项式模型的表现远优于线性模型。干得好！

让我们看看是否可以将其可视化！


In [ ]:
# Bind encoded package column to the results
poly_results <- poly_results %>% 
  bind_cols(package_encode %>% 
              rename(package_integer = package)) %>% 
  relocate(package_integer, .after = package)


# Print new results data frame
poly_results %>% 
  slice_head(n = 5)


# Make a scatter plot
poly_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_line(aes(y = .pred), color = "midnightblue", size = 1.2) +
  xlab("package")


您可以看到一条更符合您数据的曲线！🤩

您可以通过向 `geom_smooth` 传递一个多项式公式，使其更加平滑，如下所示：


In [ ]:
# Make a scatter plot
poly_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_smooth(method = lm, formula = y ~ poly(x, degree = 4), color = "midnightblue", size = 1.2, se = FALSE) +
  xlab("package")

就像一条平滑的曲线！🤩

以下是如何进行新的预测：


In [ ]:
# Make a hypothetical data frame
hypo_tibble <- tibble(package = "bushel baskets")

# Make predictions using linear model
lm_pred <- lm_wf_fit %>% predict(new_data = hypo_tibble)

# Make predictions using polynomial model
poly_pred <- poly_wf_fit %>% predict(new_data = hypo_tibble)

# Return predictions in a list
list("linear model prediction" = lm_pred, 
     "polynomial model prediction" = poly_pred)


`多项式模型`的预测是合理的，结合`价格`和`包装`的散点图来看确实如此！而且，如果这个模型比之前的模型更好，那么根据相同的数据，你需要为这些更贵的南瓜做好预算！

🏆 干得好！你在一节课中创建了两个回归模型。在回归的最后一部分，你将学习逻辑回归以确定类别。

## **🚀挑战**

在这个笔记本中测试几个不同的变量，看看相关性如何影响模型的准确性。

## [**课后测验**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/14/)

## **复习与自学**

在本课中我们学习了线性回归。还有其他重要的回归类型。阅读关于逐步回归、岭回归、套索回归和弹性网络技术的内容。一个很好的课程是[斯坦福统计学习课程](https://online.stanford.edu/courses/sohs-ystatslearning-statistical-learning)。

如果你想了解更多关于如何使用出色的Tidymodels框架，请查看以下资源：

-   Tidymodels官网：[Tidymodels入门](https://www.tidymodels.org/start/)

-   Max Kuhn 和 Julia Silge，[*Tidy Modeling with R*](https://www.tmwr.org/)*.*

###### **特别感谢：**

[Allison Horst](https://twitter.com/allison_horst?lang=en) 创作了令人惊叹的插图，使R语言更加友好和吸引人。可以在她的[画廊](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM)中找到更多插图。



---

**免责声明**：  
本文档使用AI翻译服务[Co-op Translator](https://github.com/Azure/co-op-translator)进行翻译。尽管我们努力确保翻译的准确性，但请注意，自动翻译可能包含错误或不准确之处。原始语言的文档应被视为权威来源。对于关键信息，建议使用专业人工翻译。我们对因使用此翻译而产生的任何误解或误读不承担责任。
